<a href="https://colab.research.google.com/github/vadim-privalov/Neiroset_Novosibirsk/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22Lesson_35_basic_motion_detection_ENG%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic motion detection and tracking with Python and OpenCV
# Базовое обнаружение и отслеживание движения с помощью Python и OpenCV

В этом уроке мы узнаем, как построить базовую систему обнаружения и отслеживания движения для домашнего наблюдения с использованием методов компьютерного зрения. Эта система будет работать как с предварительно записанными видео, так и с прямыми трансляциями с вашей веб-камеры.

In this tutorial, we will learn how to build a basic motion detection and tracking system for home surveillance using computer vision techniques. This system will work with both pre-recorded videos and live feeds from your webcam.



Вычитание фона имеет решающее значение во многих приложениях компьютерного зрения. Мы используем его для подсчета количества автомобилей, проезжающих через пункт взимания платы за проезд. Мы используем его для подсчета количества людей, которые входят в магазин и выходят из него. И мы используем его для обнаружения движения.

Background subtraction is critical in many computer vision applications. We use it to count the number of cars passing through a toll station. We use it to count the number of people entering and leaving a store. And we use it to detect motion.



Фон нашего видео в значительной степени остается статичным и неизменным в течение всего видео. Поэтому мы можем отслеживать его на предмет существенных изменений. Если есть существенное изменение, которое мы обнаружили - это изменение обычно соответствует движению на нашем видео.

The background of our video is pretty much static and unchanged throughout the entire video. Therefore, we can track it for significant changes. If there is a significant change that we found, that change usually corresponds to the movement in our video.



### Скачать zip-файл с кодом

### Download the code zip file

In [1]:
!wget http://dataudt.ru/datasets/cv/Lesson_35.basic_motion_detection.zip
!unzip -qq Lesson_35.basic_motion_detection.zip
%cd /content/Lesson_35.basic_motion_detection

--2022-02-08 09:20:58--  http://dataudt.ru/datasets/cv/Lesson_35.basic_motion_detection.zip
Resolving dataudt.ru (dataudt.ru)... 37.228.117.130
Connecting to dataudt.ru (dataudt.ru)|37.228.117.130|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30455740 (29M) [application/zip]
Saving to: ‘Lesson_35.basic_motion_detection.zip’

Lesson_35.basic_mot 100%[===================>]  29.04M  15.9MB/s    in 1.8s    

2022-02-08 09:21:01 (15.9 MB/s) - ‘Lesson_35.basic_motion_detection.zip’ saved [30455740/30455740]

/content/Lesson_35.basic_motion_detection


### Импорт библиотек
### Import Packages

In [2]:
# импорт библиотек
# import the necessary packages
from matplotlib import pyplot as plt
import datetime
import imutils
import time
import cv2

### Базовое обнаружение и отслеживание движения с помощью Python и OpenCV

### Basic motion detection and tracking with Python and OpenCV

In [3]:
# явно укажем аргументы необходимые для работы
# video - путь к видеофайлу
# min_area - минимальный размер области
# output - выходное видео

# explicitly specify the arguments needed to work
# video -  path to the video file
# min_area - minimum area size
# output - output video 
args = {
	"video": "videos/video.mp4",
	"min_area": 1000,
	"output" : "output.avi"
}

In [4]:
# загружаем видео и указываем путь к выходному файлу
# grab a reference to the video file and initialize pointer to output
# video file
print("[INFO] opening video file...")
vs = cv2.VideoCapture(args["video"])
writer = None

# инициализируем первый кадр видео
# initialize the first frame in the video stream
firstFrame = None

[INFO] opening video file...


In [5]:
# перебираем кадры из видео
# loop over the frames of the video
while True:
    # захватываем текущий кадр и инициализирум текст "occupied/unoccupied"
	# grab the current frame and initialize the occupied/unoccupied
	# text
	frame = vs.read()[1]
	text = "Unoccupied"

    # если мы не захватили кадр, то мы дошли до конца видео
	# if the frame could not be grabbed, then we have reached the end
	# of the video
	if frame is None:
		break

    # изменяем размер кадра, преобразоваем его в оттенки серого и размываем
	# resize the frame, convert it to grayscale, and blur it
	frame = imutils.resize(frame, width=500)
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
	gray = cv2.GaussianBlur(gray, (21, 21), 0)

    # если нет первого кадра, то инициализируем его
	# if the first frame is None, initialize it
	if firstFrame is None:
		firstFrame = gray
		continue

    # вычисляем абсолютную разницу между текущим кадром и
	# первым кадром
	# compute the absolute difference between the current frame and
	# first frame
	frameDelta = cv2.absdiff(firstFrame, gray)
	thresh = cv2.threshold(frameDelta, 25, 255, cv2.THRESH_BINARY)[1]

    # расширяем пороговое изображение, чтобы заполнить пространство, затем находим контуры
	# на пороговом изображении
	# dilate the thresholded image to fill in holes, then find contours
	# on thresholded image
	thresh = cv2.dilate(thresh, None, iterations=2)
	cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
		cv2.CHAIN_APPROX_SIMPLE)
	cnts = imutils.grab_contours(cnts)

    # перебираем контуры
	# loop over the contours
	for c in cnts:
        # если контур слишком маленький, игнорируем его
		# if the contour is too small, ignore it
		if cv2.contourArea(c) < args["min_area"]:
			continue

        # вычисляем ограничивающую рамку для контура, рисуем его на кадре,
		# и обновляем текст
		# compute the bounding box for the contour, draw it on the frame,
		# and update the text
		(x, y, w, h) = cv2.boundingRect(c)
		cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
		text = "Occupied"

    # рисуем текст и дату на кадре
	# draw the text and timestamp on the frame
	cv2.putText(frame, "Room Status: {}".format(text), (10, 20),
		cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
	cv2.putText(frame, datetime.datetime.now().strftime("%A %d %B %Y %I:%M:%S%p"),
		(10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)

    # если нет видео * И * мы должны записать
    # выходное видео на диск, то инициализируем камеру
	# if the video writer is None *AND* we are supposed to write
	# the output video to disk initialize the writer
	if writer is None and args["output"] is not None:
		fourcc = cv2.VideoWriter_fourcc(*"MJPG")
		writer = cv2.VideoWriter(args["output"], fourcc, 20,
			(frame.shape[1], frame.shape[0]), True)
  
    # если видео есть, записываем кадр с распознанным лицом на диск
	# if the writer is not None, write the frame with recognized
	# faces to disk
	if writer is not None:
		writer.write(frame)

# очищаем данные камеры
# do a bit of cleanup
vs.release()

# проверяем, нужно ли очистить данные
# check to see if the video writer point needs to be released
if writer is not None:
	writer.release()

Cell execution may take some time. Our output video is created in .avi format. We need to convert it to .mp4 format.

Выполнение ячейки может занять некоторое время. Наше выходное видео создается в формате .avi. Нам нужно преобразовать его в формат .mp4.

In [6]:
!ffmpeg -i output.avi output.mp4


ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [7]:
#@title Воспроизводим выходное видео
#@title Display video inline

from IPython.display import HTML
from base64 import b64encode

mp4 = open("output.mp4", "rb").read()
dataURL = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % dataURL)